In [2]:
import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("balanced-all.csv")
df.head()

,filename,gender
0,data/cv-other-train/sample-069205.npy,female
1,data/cv-valid-train/sample-063134.npy,female
2,data/cv-other-train/sample-080873.npy,female
3,data/cv-other-train/sample-105595.npy,female
4,data/cv-valid-train/sample-144613.npy,female


In [5]:
df.tail()

,filename,gender
66933,data/cv-valid-train/sample-171098.npy,male
66934,data/cv-other-train/sample-022864.npy,male
66935,data/cv-valid-train/sample-080933.npy,male
66936,data/cv-other-train/sample-012026.npy,male
66937,data/cv-other-train/sample-013841.npy,male


In [6]:
n_sample=len(df)
n_male_sample=len(df[df['gender']=='male'])
n_female_sample=len(df[df['gender']=='female'])
print("total samples :",n_sample)
print("total male sample:",n_male_sample)
print("total female sample:",n_female_sample)

total samples : 66938
total male sample: 33469
total female sample: 33469


In [7]:
import numpy as np
import os
import tqdm

In [8]:
lablel2int={
    "male":1,
    "female":0
}

def load_data(vector_length=128):
    if not os.path.isdir("results"):
        os.mkdir("results")
    if os.path.isfile("results/features.npy") and os.path.isfile("results/lables.npy"):
        X=np.load("results/features.npy")
        y=np.load("results/labels.npy")
        return X,y
    df=pd.read_csv("balanced-all.csv")
    n_sample=len(df)
    n_male_sample=len(df[df['gender']=='male'])
    n_female_sample=len(df[df['gender']=='female'])
    print("total samples :",n_sample)
    print("total male sample:",n_male_sample)
    print("total female sample:",n_female_sample)
    
    X=np.zeros((n_sample, vector_length))
    y=np.zeros((n_sample,1))
    
    for i,(filename,gender) in tqdm.tqdm(enumerate(zip(df['filename'],df['gender'])),"Loding data",total=n_sample):
        features=np.load(filename)
        X[i]=features
        y[i]=lablel2int[gender]
        
    np.save("results/features",X)
    np.save("results/labels",y)
    return X,y
    
        
    

In [17]:
load_data()

total samples : 66938
total male sample: 33469
total female sample: 33469


Loding data: 100%|███████████████████████████████████████████████████████████████| 66938/66938 [12:07<00:00, 91.96it/s]


(array([[8.33691835e-01, 1.84069288e+00, 7.21150815e-01, ...,
         2.22568936e-03, 3.72419163e-04, 4.14350980e-05],
        [3.44792323e-04, 3.78249679e-04, 7.02428690e-04, ...,
         5.25400345e-08, 2.83253740e-08, 2.14376583e-09],
        [1.90136401e-04, 2.43997492e-04, 4.60427953e-04, ...,
         1.14822782e-04, 2.32082821e-05, 1.38619748e-06],
        ...,
        [1.16413343e-03, 9.43325169e-04, 1.52071775e-03, ...,
         3.66348862e-09, 1.36717937e-09, 1.47780455e-10],
        [6.36986643e-02, 1.34973586e-01, 2.42811322e+00, ...,
         3.79042875e-04, 8.72957244e-05, 6.89193166e-06],
        [1.45035911e+00, 1.17254317e+00, 2.72492021e-01, ...,
         2.80375849e-03, 1.14007434e-03, 1.97923204e-04]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]]))

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
def split_data(X,y,test_size=0.1,valid_size=0.1):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_size,random_state=7)
    X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=valid_size,random_state=7)
    return{
        "X_train":X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }

In [12]:
X,y=load_data()
data=split_data(X,y,test_size=0.1,valid_size=0.1)

total samples : 66938
total male sample: 33469
total female sample: 33469


Loding data: 100%|███████████████████████████████████████████████████████████████| 66938/66938 [15:09<00:00, 73.60it/s]


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping



In [14]:
def create_model(vector_length=128):
    model=Sequential()
    model.add(Dense(256,input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")
    model.summary()
    return model
    

In [15]:
model=create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

In [25]:
tensorboard=TensorBoard(log_dir="logs")
early_stopping=EarlyStopping(mode="min",patience=5,restore_best_weights=True)

batch_size=64

epochs=100

model.fit(data["X_train"],data["y_train"],epochs=epochs,batch_size=batch_size,validation_data=(data["X_valid"],data["y_valid"]),
         callbacks=[tensorboard,early_stopping])

Epoch 1/100
848/848 [==============================] - 15s 13ms/step - loss: 0.5495 - accuracy: 0.7745 - val_loss: 0.3645 - val_accuracy: 0.8576
Epoch 2/100
848/848 [==============================] - 11s 13ms/step - loss: 0.4077 - accuracy: 0.8390 - val_loss: 0.3389 - val_accuracy: 0.8657
Epoch 3/100
848/848 [==============================] - 10s 12ms/step - loss: 0.3784 - accuracy: 0.8523 - val_loss: 0.3061 - val_accuracy: 0.8825
Epoch 4/100
848/848 [==============================] - 10s 12ms/step - loss: 0.3606 - accuracy: 0.8602 - val_loss: 0.3045 - val_accuracy: 0.8815
Epoch 5/100
848/848 [==============================] - 10s 12ms/step - loss: 0.3452 - accuracy: 0.8690 - val_loss: 0.2921 - val_accuracy: 0.8915
Epoch 6/100
848/848 [==============================] - 10s 12ms/step - loss: 0.3306 - accuracy: 0.8740 - val_loss: 0.2864 - val_accuracy: 0.8903
Epoch 7/100
848/848 [==============================] - 10s 12ms/step - loss: 0.3283 - accuracy: 0.8768 - val_loss: 0.2758 - val_ac

In [23]:
# Save the model in the native Keras format
model.save("results/model.keras")
model.save("results/model.h5")

C:\Users\Riya\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss,accuracy=model.evaluate(data["X_test"],data["y_test"],verbose=1)

print(f"Loss:{loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Evaluating the model using 6694 samples...
210/210 [==============================] - 1s 5ms/step - loss: 0.2227 - accuracy: 0.9183
Loss:0.2227
Accuracy: 91.83%


In [27]:
try:
    import librosa
    print("librosa is installed.")
except ImportError:
    print("librosa is not installed.")



librosa is installed.


In [30]:
import librosa
import numpy as np

def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
        result = np.hstack((result, tonnetz))
    return result


In [31]:
import pyaudio
import wave


In [1]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
OUTPUT_FILE = "recorded_voice.wav"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open the audio stream
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

print("Recording started. Speak into the microphone...")

frames = []

# Record audio for the specified duration
for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording completed.")

# Stop and close the audio stream
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded audio to a WAV file
with wave.open(OUTPUT_FILE, "wb") as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))

print(f"Saved recorded voice to '{OUTPUT_FILE}'.")


Recording started. Speak into the microphone...
Recording completed.
Saved recorded voice to 'recorded_voice.wav'.


In [32]:
file_name="C:/Users/Riya/OneDrive/Desktop/voice detection/gender-recognition-by-voice/recorded_voice.wav"
features = extract_feature(file_name, mel=True).reshape(1, -1)
male_prob = model.predict(features)[0][0]
female_prob = 1 - male_prob
gender = "male" if male_prob > female_prob else "female"

# show the result!
print("Result:", gender)
print(f"Probabilities:     Male: {male_prob * 100:.2f}%    Female: {female_prob * 100:.2f}%")

1/1 [==============================] - 0s 281ms/step
Result: male
Probabilities:     Male: 88.72%    Female: 11.28%


In [38]:
from tkinter import *
from tkinter import messagebox
from PIL import ImageTk,Image
root=Tk()


root.title("Voice-based gender detection")
root.geometry("600x400")
root.minsize(300,300)
root.maxsize(700,700)

root.iconbitmap("D:\\tkinter\\logo.ico")
root.configure(bg="lightblue")

header=Label(root,text="Voice based gender identification",bg="lightblue",
            foreground="black",font=("Arial",24,"bold"))
header.pack(pady=10)
# root.wm_attributes('-transparentcolor',"purple")

def record():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = 5
    OUTPUT_FILE = "recorded_voice.wav"

    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Open the audio stream
    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

    print("Recording started. Speak into the microphone...")

    frames = []

    # Record audio for the specified duration
    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording completed.")

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded audio to a WAV file
    with wave.open(OUTPUT_FILE, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b"".join(frames))

    print(f"Saved recorded voice to '{OUTPUT_FILE}'.")
button1=Button(text="Record voice",bg="lightblue",activebackground="blue",font=("Arial",18,"bold"),
      borderwidth=3,command=record)
button1.pack(pady=30)
predict_label=Label(root,text="",bg="lightblue",font=("Arial",15,"bold"))
predict_label.pack()

def predict_gender():
    file_name="C:/Users/Riya/OneDrive/Desktop/voice detection/gender-recognition-by-voice/recorded_voice.wav"
    features = extract_feature(file_name, mel=True).reshape(1, -1)
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"

    # show the result!
    print("Result:", gender)
    print(f"Probabilities:     Male: {male_prob * 100:.2f}%    Female: {female_prob * 100:.2f}%")
    predict_label.config(text="Predicted Gender :"+gender)

button2=Button(text="Gender",bg="lightblue",activebackground="blue",font=("Arial",18,"bold"),
      borderwidth=3,command=predict_gender)
button2.pack(pady=25)

root.mainloop()

no
